In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql
import time
import jieba
import jieba.analyse

link=pymysql.connect(
    host="localhost",
    user="root",
    passwd="",
    db="2018-12-06")

cmd=link.cursor()

url="https://www.techbang.com/tags/6117?page="
page=1

while True:
    d=requests.get(url+str(page))
    p=BeautifulSoup(d.text,"html.parser")
    q=p.find("article",{"class":"articles"})

    for a in q.find_all("article"):
        aa=a.find("a")
        d2=requests.get(aa['href'])
        p2=BeautifulSoup(d2.text,"html.parser")	

        title=p2.find("h1") #.find("h1").text.strip()
        if title !=None:
            title = title.text.strip()
        else:
            title = ""

        p3=p2.find_all("section",{"itemprop":"articleBody"}) #取得內文框
        c=p2.find_all("p",{"class":"excerpt"})

        writer=p2.find("div",{"class":"post-header-additional"})
        if writer != None:
            writer = writer.find("a",{"class":"nickname"}).text
        else:
            writer = ""

        tm=p2.find("div",{"class":"post-header-additional"})
        if tm !=None:
            tm = tm.find("a",{"class":"nickname"}).findNext('span').text
        else:
            tm = ""

        tag=p2.find("p",{"class":"article-tags"})
        if tag !=None:
            tag = tag.text.strip()
        else:
            tag = ""

        for cc in c:  #excerpt
            content1=cc.text.strip()

        content3=""
        d=p2.select("#article-content p")
        for c in d:
            content33=c.text.strip()
            content3+=content33
        kw=jieba.analyse.extract_tags(content3)

        pm={"title":title,
        "writer":writer,
        "date":tm[4:15],
        "create_time":tm[16:21],
        "excerpt":content1,
        "content":content3,
        "tag":tag[3:],
        "extract_tags":",".join(kw)
        }

        cmd.execute("INSERT INTO `new`(`title`,`writer`,`date`,`create_time`,`excerpt`,`content`,`tag`,`extract_tags`) VALUES(%(title)s,%(writer)s,%(date)s,%(create_time)s,%(excerpt)s,%(content)s,%(tag)s,%(extract_tags)s)"
            ,pm)
        link.commit()

    page+=1
    time.sleep(3)
    if page==8:
        break
link.close()